# Direct minimisation

In this section we return our attention to the DFT minimisation problem
$$ \min_{P\in\mathcal{P}} \mathcal{E}(P). $$
Instead of solving the SCF problem, i.e. focusing on satisfying its first-order optimality condition, a mathematically more natural approach to directly develop methods to solve this minimisation problem. This is referred to as **direct minimisation** (DM).

For simplicity we restrict the discussion of DM to the case of infinite $\beta$ or zero temperature, where all occupations $f_i$ are either $0$ or $1$.
In this case a reformulation of the minimisation problem in terms of the orbitals is trivial, since 
$$ P = \sum_i^{N} |\phi_i\rangle \langle \phi_i|. $$
where $N$ is the number of electrons. 
Therefore one may alternatively solve 
$$ \displaystyle \text{min}_{\{\psi_i\}} \mathcal{E}_\text{DFT}(\{\psi_i\}) $$
where the $\{\psi_i\}$ are constrained to be orthonormal orbitals. For non-zero temperature other occupations are possible, which overall complicates the picture and is not discussed here.


Due to the orthogonality constraints on the orbitals $\{\psi_i\}$,
  the set of admissible orbitals does not form a vector space,
  but much rather the unknowns of $\{\psi_i\}$ belong to a Stiefel manifold.
  This needs to be taken into account
  (via appropriate projections) in order to get the correct minimum
in a minimisation procedure. For more details, see for example [A. Edelman, T. Arias, S. Smith *SIAM J. Mat. Anal. Appl.* **20**, 303 (1998) DOI 10.1137/s0895479895290954](http://dx.doi.org/10.1137/s0895479895290954).

The gradient of $\mathcal{E}_0$ wrt. orbitals is easily obtained via the chain rule from our previous developments:
  $$ \left\langle \phi \middle| \nabla_{\psi_i} \mathcal{E}_0\left(\sum_i |\psi_i\rangle \langle \psi_i |\right) \right\rangle = \langle \phi | \nabla_P \mathcal{E}_0(P) \ \psi_i\rangle 
  + \langle \psi_i | \nabla_P \mathcal{E}_0(P) \ \phi \rangle
  = 2 \langle \phi | H_\text{KS} \psi_i \rangle$$

To show this in practice we proweed to a simple implementation using `DFTK.jl` for the energy functional of the density-functional theory problem and `Optim.jl` to do the minimisation on the Stiefal manifold. To keep the implementation simple we further restrict ourselves to a single $k$-Point. Going beyond that requires a little more bookkeeping
  (see the [DFTK implementation](https://github.com/JuliaMolSim/DFTK.jl/blob/master/src/scf/direct_minimization.jl)).

In [1]:
using DFTK
using Optim
using LineSearches

# Standard silicon setup
a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]
model  = model_LDA(lattice, atoms)
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[1, 1, 1]);

# One unit cell has 2 Silicon atoms.
# In the model we use (where only valence electrons are explictly treated)
# this makes 8 electrons, which requires 4 bands with 2 electrons each:
occupation = [2.0, 2.0, 2.0, 2.0]

# We specify a random initial guess for the 4 orbitals:
n_G = length(G_vectors(only(basis.kpoints)))
ψ0 = Matrix(qr(randn(ComplexF64, n_G, 4)).Q);

# Function to compute energies and gradients
function fg!(E, G, ψ)
    ρ = compute_density(basis, [ψ], [occupation])
    energies, H = energy_hamiltonian(basis, [ψ], [occupation]; ρ=ρ)

    if G !== nothing
        # Optim expects the gradient in G
        occupation_ψ = 2.0  # Hard-coded occupation of orbital ψ 
        G .= 2 * occupation_ψ * (H.blocks[1] * ψ)
    end
    energies.total
end

# Select a quasi-Newton algorithm with backtracking linesearches
# to avoid to many costly gradient evaluations
algorithm = Optim.LBFGS(manifold=Optim.Stiefel(),
                        linesearch=LineSearches.BackTracking())

# Set some convergence options in Optim:
options = Optim.Options(; allow_f_increases=true, show_trace=true, x_tol=1e-6)

# Run the direct minimisation
res = Optim.optimize(Optim.only_fg!(fg!), ψ0, algorithm, options)

@show res.minimum

LoadError: MethodError: no method matching model_LDA(::Matrix{Float64}, ::Vector{Pair{ElementPsp, Vector{Vector{Float64}}}})
[0mClosest candidates are:
[0m  model_LDA(::AbstractMatrix, [91m::Vector{<:DFTK.Element}[39m, [91m::Vector{<:AbstractVector}[39m; kwargs...) at ~/.julia/packages/DFTK/rFq5C/src/standard_models.jl:51
[0m  model_LDA([91m::AtomsBase.AbstractSystem[39m, ::Any...; kwargs...) at ~/.julia/packages/DFTK/rFq5C/src/external/atomsbase.jl:94

Finally let us note that DM in combination with metals (also known as *ensemble density-functional theory*) is tricky. In this case there is no band gap and thus finite $\beta$ is usually taken. Furthermore there are possible degeneracies of the orbitals at the Fermi level ($i = N$).
  This problem manifests when computing the gradient of $\mathcal{E}_\text{DFT}$
  in such a setting, where thus special care is needed to not run into
  "division by zero" issues.

## Comparing SCF and DM methods

In this section we want to elaborate on the relationship between self-consistent-field and direct minimisation methods. For this we return to the finite-temperature setting in the grand-canonical ensemble and use the formulation in terms of density matrices
$$ \min_{P\in\mathcal{P}} \mathcal{E}(P). $$
where 
$$ \mathcal{E}(P) = \mathcal{E}_0(P) - \frac{1}{\beta} \mathrm{Tr}[s(P)] - \mu \mathrm{Tr}(P). $$
and
$$\begin{align*}
    \mathcal{H} = \left\{H \in \mathbb{C}^{N_b \times N_b}, H^\dagger = H \right\}.
    \mathcal{P} = \left\{P \in \mathcal{H}, 0 < P < 1 \right\}.
\end{align*}$$

The gradient of $\mathcal{E}_0$ we already identified as $ H_\text{KS}(P) = \nabla \mathcal{E}_0(P)$. In analogy to the second derivative of $g(\rho)$ we denote the hessian of $\mathcal{E}_0$ as
$$ \underline{K}(P) = \underline{d^2 \mathcal{E}_0}(P) = \underline{d \nabla \mathcal{E}_0}(P), $$
where we use an underline denote this object as a
**super-operators**, i.e. operators from $\mathcal{H}$ to $\mathcal{H}$.
Following from the analogy we also refer to $\underline{K}$ as the "four-point" kernel.

Similarly we can obtain the four-point analogon to $\chi_0 = D'$ as the derivative $\underline{\chi_0} = d f_\text{FD}$ computed as
$$\begin{align*}
      &\underline{\chi_{0}}\left(\sum_{i=1}^{N_{\rm b}} \varepsilon_{i} |\phi_{i}\rangle\langle \phi_{i}|\right) \cdot \delta H = \sum_{i=1}^{N_{\rm b}}\sum_{j=1}^{N_{\rm b}} \frac{f_\text{FD}(\varepsilon_{i}) - f_\text{FD}(\varepsilon_{j})}{\varepsilon_{i}-\varepsilon_{j}}\langle  \phi_{i}, \delta H \phi_{j} \rangle |\phi_{i}\rangle\langle \phi_{j}|
\end{align*}$$
(for details see [M. Herbst, A. Levitt *J. Comput. Phys.* **459**, 111127 (2022)](http://dx.doi.org/10.1016/j.jcp.2022.111127)). Comparing with the Adler-Wiser formula makes the analogy to $\chi_0$ particularly apparent.


Further we will need the gradient and Hessian of the free energy $\mathcal{E}$ for our analysis.
We recall the gradient of $\mathcal{E}$ as
$ \nabla \mathcal{E}(P) = H_\text{KS}(P) - \mu - \frac1{\beta} s'(P). $
To obtain the Hessian we differentiate once more
  $$
  \underline{d^2 \mathcal{E}}(P) \cdot \delta P
  = \underline{K}(H_\text{KS}(P)) \cdot \delta P
  - \frac{1}{\beta}
  \sum_{i=1}^{N_b} \sum_{j=1}^{N_b}
  \frac{s'(p_i) - s'(p_j)}{p_i - p_j}
  |\phi_i \rangle \langle \phi_i | \delta P \phi_j\rangle
  \langle \phi_j |
  .$$
Defining
  $$
  \underline{\Omega}\left(
      \sum_{i=1}^{N_b} \varepsilon_i \phi_i \rangle \langle \phi_i |
  \right) \cdot \delta P
  = - \sum_{i=1}^{N_b} \sum_{j=1}^{N_b}
  \frac{\varepsilon_i - \varepsilon_j}{f_\text{FD}(\varepsilon_i) - f_\text{FD}(\varepsilon_j)}
  |\phi_i \rangle \langle \phi_i | \delta P \phi_j\rangle
  \langle \phi_j |
  $$
  we write this more compactly as
  $$
  \underline{d^2 \mathcal{E}}(P)
  = \underline{K}(H_\text{KS}(P)) + \underline{\Omega}(f_\text{FD}^{-1}(P)).
  $$

Comparing the expressions for $\underline{\chi_0}$ and $\underline{\Omega}$ we furthermore note for later convenience that
$$ \underline{\Omega} = - \underline{\chi_0}^{-1}.$$

### Direct minimisation in density-matrix formalism

As an example for a direct minimisation approach we will consider **damped gradient descent**
$$ P_{n+1} = P_n - \alpha \nabla \mathcal{E}(P_n). $$

If $\alpha$ is chosen small enough, this iteration converges to a local minimiser $P_\ast$, which we assume to be non-degenerate. To understand the convergence we denote the error in each iteration as $E_n = P_n - P_\ast$ and write
$$\begin{align*}
E_{n+1} &= E_n - \alpha \nabla \mathcal{E}(P_\ast + E_n) \\
&= E_n - \alpha [\nabla \mathcal{E}(P_\ast) + \underline{d^2 \mathcal{E}}(P_\ast) \cdot E_n ] \\
&= \left[1 - \alpha (\underline{K}_\ast + \underline{\Omega}_\ast)\right] E_n,
\end{align*}$$
where $|_\ast$ denotes that these quantities are evaluated at the fixed-point density matrix $P_\ast$ or Hamiltonian $H(P_\ast)$.

Similar to our discussion in the previous notebook the asymptotic rate of convergence is related to the eigenvalues of
$1 - \alpha J_\text{grad}$
$$J_\text{grad} = \underline{K}_\ast + \underline{\Omega}_\ast.$$
In particular with $\alpha$ chosen optimally it is the spectral condition number $\kappa = \frac{\lambda_\text{max}}{\lambda_\text{min}}$ which dictates the rate of convergence.

### SCF in density-matrix formalism

The fixed-point problem
$$ P = f_\text{FD}(H_\text{KS}(P)) $$
in the density-matrix formalism of DFT is fully analogous
to the problem
$$ \rho = D(V(\rho))$$
underlying the density-mixing SCF algorithm we discussed in the previous notebook. Using the appropriate four-point analoges to $\chi_0$ and $K$ our considerations wrt. convergence of density mixing can therefore be directly adapted to a density-matrix mixing algorithm.

In particular the convergence of the damped iterations
$$ P_{n+1} = P_n + \alpha [f_\text{FD}(H_\text{KS}(P_n)) - P_n]$$
are characterised by the eigenvalues of $1 - \alpha J_\text{SCF}$
with
$$ J_\text{SCF} = 1 - \left. \underline{\chi_0}\right|_\ast \underline{K}_\ast = 1 + \underline{\Omega}_\ast^{-1} \underline{K}_\ast, $$ 
where $|_\ast$ denotes that these quantities are evaluated at the fixed-point density matrix $P_\ast$ or Hamiltonian $H(P_\ast)$.

### Comparison of convergence analysis

Both methods have a very similar structure, differing only in the matrix $J$ of the Jacobian $1 - \alpha J$. Most notably is the appearance of $\underline{\Omega}_\ast$ as a summand in $J_grad$ for gradient descent and as an *inverse* in $J_\text{SCF}$.

To keep the discussion here simple, we will only give a handwavy argument based on the extremal eigenvalues of $\underline{\Omega}_\ast$. For a more detailed discussion for the case of insulators at zero temperature see [E. Cances, G. Kemlin, A. Levitt *SIAM J. Mat. Anal. Appl.* **42**, 243 (2021) DOI 10.1137/20m1332864](http://dx.doi.org/10.1137/20m1332864).

The largest eigenvalue of $\underline{\Omega}_\ast$ depends on $\varepsilon_{N_b} - \varepsilon_1$, i.e. the difference between the largest and smallest eigenvalue of the Kohn-Sham Hamiltonian, which becomes larger and larger as the discretisation is improved. While this is not a big problem for SCF methods, this can become an issue for gradient descent in theory. However, in practice it can be cured by appropriate preconditioning.

If the smallest eigenvalue of $\underline{\Omega}_\ast$ is close to zero, this gives rise to a large eigenvalue in $J_\text{SCF}$, while the smallest eigenvalue of $J_\text{grad}$ stays bounded due to the $\underline{K}$ term. While small eigenvalues in $\underline{\Omega}_\ast$ are thus a problem for SCF methods they are not for direct minimisation methods.

How can such small gaps arise? Assume one only uses a very small temperature, such that $\beta$ is large and $f_\text{FD}$ is pretty much a step function. In this case a good approximate lower bound to the smallest eigenvalue of $\underline{\Omega}_\ast$ is the gap $\nu = \varepsilon_a - \varepsilon_i$, where $a$ denotes the first orbital above the Fermi level ($\varepsilon_a > \mu$) and $i$ the last orbital below the Fermi level ($\varepsilon_i < \mu$). A closing gap $\nu$ thus causes the smallest eigenvalue of $\underline{\Omega}_\ast$ to decrease as well.
If one wants to stick with SCF methods the remedy is increase the temperature (use a smaller $\beta$). For metallic systems, where the gap is zero, this always has to be done. For metals with $\nu=0$ the smallest eigenvalue of $\underline{\Omega}_\ast$ is bounded from below by $-1 / f'_\text{FD}(\mu) = 4/\beta$, such that a large enough temperature can ensure convergence of the SCF.

Whether SCF or direct minimisation should be employed in practice overall depends not only on the considerations of the convergence rate outlined here, but also questions related to the computational cost of each step, the effectiveness of convergence acceleration schemes and the robustness of iterations needs to be considered. In any case the mathematical anlysis outlined here, suggests that direct minimisation methods should be beneficial in particular for modelling insulators with smaller gaps.